# Final Project (Group 3)

## Topic: Can you stop the run?

## I. Intro

   It's the Bottom 9th inning in the final match of **NTU87 Baseball Golden Championship**  
   Now, the ballgame is tied at *7-7*, but your closer isn't performing well today.  
   All the bases are loaded, and still 1 out to go......
     
   The next batter is on the plate, and he hits the first pitch.    
   It's a **Penetrating Ground Ball** carrying sooooo much speed. Apparently, your infielders are unable to stop it.  
   Assume you're the best outfielder on your team, and you've intercepted the ball flawlessly.  
   
   However, you must **throw the ball fast and accurately enough** back to the homeplate,  
   to block their winning run and extend the match.  
   
   **Can you be the hero of your team?**
   
## II(a). Use of Materials Taught in the Course: Coding

### Python statement covered in this project

* `While`
* `Random`
* `If-else branch`
* `functions`

### VPython statement covered in this project

* `vector`
* `box`, `sphere`, `arrow`
* `rate`
* `widgets`
* `camera`

Reference: [Jupyter VPython Documentation](http://www.glowscript.org/docs/VPythonDocs/index.html)


## II(b). Use of Materials Taught in the Course: Physics

* Momentum Principle  
* Projectile Movement
* Air-resistance
* Magnus effect
  
References:
   1. [2D Rotation Matrix](http://mathworld.wolfram.com/RotationMatrix.html)
   2. [Air resistance](http://dynref.engr.illinois.edu/afp.html)
   3. [Drag coefficient of a sphere](https://en.wikipedia.org/wiki/Drag_coefficient)
   4. [Launch speed & Spinrate](https://www.drivelinebaseball.com/2016/11/spin-rate-what-we-know-now/)
   5. [Magnus effect on the ball](http://spiff.rit.edu/richmond/baseball/traj_may2011/traj.html)
   6. [COR of a baseball](http://twbsball.dils.tku.edu.tw/wiki/index.php/%E7%A1%AC%E5%BC%8F%E6%A3%92%E7%90%83)


## III. Objective

 **在三壘跑者跑回本壘得分之前，身為外野手的你必須將球傳到捕手的手上**  


## IV. Method
 1. **設定風速風向及跑者位置**
 2. **調整投球力道、仰角及水平偏角**

## V. Results
 成功條件：在跑者抵達本壘之前成功將球送至捕手所在位置**半徑一公尺，高度兩公尺的圓柱空間**

## Code: Part A
 ### Just have fun playing with it!!! 

 Random: **wind**  
 Fixed: **runner's speed**  
 User-defined: **outfielder's position, all launching conditions**  
 

In [1]:
from vpython import *
from random import random
import numpy as np

## Define Constants

density = 1.225 # air density
Cd = 0.47 # drag coefficient of a sphere
pi = 3.1415926

g=9.8 # g = 9.8 m/s^2
size = 3.68e-2 # Baseball radius = 3.68 cm
mass = 0.145 # Baseball mass = 0.145 kg
height = 2.0 # Where the ball leaves your hand
Area = pi * (size**2) # effective surface area
density_ball = mass / (4/3 * size**3 * pi)

COR = 0.57 #Coefficient of Restitution of a baseball(regulated)

Cl = 0.22 # Lift coefficient (used in calculating Magnus force)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# open the ballpark
scene = canvas(width=1000, height=400, center=vector(-20, 30, 20),background=vector(0,0,0), title = 'Click to place yourself')
scene.camera.pos = vector(-20, 100, 20)
scene.camera.axis = vector(20, -100, -20)
field = box(length=100, height=0.01, width=100, pos = vector(0, 0, 0), color=vector(0.5, 1, 0), texture = "grass.jpg") # the field
infield = box(length=40, height=0.01, width=40, pos = vector(30, 0.1, -30), color=vector(1, 0.7, 0.2), texture = "dirt.jpg") # the infield
ball = sphere(radius = 3*size, color=color.white, make_trail=True) # the baseball

catcher = box(length=0.6, height=2, width=0.6, pos = vector(48, 1, -48), color = color.cyan)
# runner is set to cover 25 meters before scoring
runner = box(length=0.6, height=2, width=0.6, pos = vector(48, 1, -23), color = color.yellow, texture = {'file':"abe.jpg", 'place':['sides']})



# add the effect of wind (using random numbers)
wind_x = random()
wind_z = random()
wind_speed = 10*random()
posneg = random()

wind = wind_speed*norm(vector(wind_x, 0, wind_z))
if(posneg <= 0.25):
    wind = wind_speed*norm(vector(-wind_x, 0, -wind_z))
elif(posneg <= 0.50):
    wind = wind_speed*norm(vector(-wind_x, 0, wind_z))
elif(posneg <= 0.75):
    wind = wind_speed*norm(vector(wind_x, 0, -wind_z))
    
wind_arrow = arrow(pos=vector(0, 20, 0), 
               axis = 3*wind, shaftwidth = 0.5, color = color.magenta)
scene.append_to_caption('Windspeed = {:1.2f} m/s\n'.format(wind_speed))



# place yourself (the hero outfielder)
scene.waitfor('click')
you = box(length=0.6, height=2, width=0.6, pos = vector(scene.mouse.pos.x, 1, scene.mouse.pos.z), color = color.cyan)
ball.pos = you.pos + vector(0, 1, 0)
scene.center = you.pos
scene.autoscale = True
print('You are {:1.2f} meters away from your cather'.format(mag(catcher.pos - you.pos)))
scene.title = 'Now, determine the angle and velocity to throw the ball back; then, click to start'

init_dir = norm(catcher.pos - you.pos) # the direction straight to your catcher
direction = init_dir # the direction you can adjust
Launch_angle = pi/6 # set default launch angle to 30 degrees
Horizontal_adjustment = 0 # the horizontal declination with respect to your catcher
Launch_v = 75*1.609*5/18 # set default launch speed to 75mph

##　Add the arrow representing launching conditions
ball_dir=arrow(pos=ball.pos, 
               axis = 0.3*Launch_v*norm( vector(direction.x, sqrt(direction.x**2 + direction.z**2)*tan(Launch_angle), direction.z) ),
               shaftwidth = ball.radius*3, color = color.red)



## Sliders to adjust the launch

# Launch angle
def setlaunch():
    wt1.text = '  Set Launch Angle : {:1d} degrees'.format(sv.value)
    direction = vector( init_dir.x*cos(sh.value/180*pi)-init_dir.z*sin(sh.value/180*pi), 0, init_dir.x*sin(sh.value/180*pi)+init_dir.z*cos(sh.value/180*pi))
    Launch_angle = sv.value/180*pi
    ball_dir.axis = 0.3*Launch_v*norm( vector(direction.x, sqrt(direction.x**2 + direction.z**2)*tan(sv.value/180*pi), direction.z) )

# Horizontal declination (using 2D Rotation Matrix to deal with the computation)
def setdec():
    wt2.text = '  Set Horizontal declination (w.r.t. catcher) : {:1.1f} degrees'.format(sh.value)
    direction = vector( init_dir.x*cos(sh.value/180*pi)-init_dir.z*sin(sh.value/180*pi), 0, init_dir.x*sin(sh.value/180*pi)+init_dir.z*cos(sh.value/180*pi))
    ball_dir.axis = 0.3*Launch_v*norm( vector(direction.x, sqrt(direction.x**2 + direction.z**2)*tan(sv.value/180*pi), direction.z) )

# Launch speed
def setspeed():
    wt3.text = '  Set Launch Speed : {:1.1f} mph'.format(ss.value)
    Launch_v = ss.value*1.609*5/18
    direction = vector( init_dir.x*cos(sh.value/180*pi)-init_dir.z*sin(sh.value/180*pi), 0, init_dir.x*sin(sh.value/180*pi)+init_dir.z*cos(sh.value/180*pi))
    ball_dir.axis = 0.3*Launch_v*norm( vector(direction.x, sqrt(direction.x**2 + direction.z**2)*tan(sv.value/180*pi), direction.z) )
    
sv = slider(min=0, max=90, value=30, step = 1, length=250, bind=setlaunch, align = 'left')
wt1 = wtext(text='  Set Launch Angle : {:1d} degrees'.format(sv.value))
scene.append_to_caption('\n')

sh = slider(min=-10, max=10, value=0, step = 0.1, length=250, bind=setdec, align = 'left')
wt2 = wtext(text='  Set Horizontal declination (w.r.t. catcher) : {:1.1f} degrees'.format(sh.value))
scene.append_to_caption('\n')

ss = slider(min=50, max=100, value=75, step = 0.1, length=250, bind=setspeed, align = 'left')
wt3 = wtext(text='  Set Launch Speed : {:1.1f} mph'.format(ss.value))
scene.append_to_caption('\n')


#RUN !!!
scene.waitfor('click')

direction = norm(ball_dir.axis)
Launch_v = ss.value*1.609*5/18
ball_p = mass*Launch_v*direction
spin_rate = 20*ss.value + 400  #RPM (we used a linear regression formula)
v_ang = (spin_rate / 60 * 2 * pi) * norm(cross(vector(0, -1, 0), ball_p/mass))

# default runner's speed = 7.5 m/s
runner_v = 7.5*norm( catcher.pos - runner.pos )
t = 0
dt = 0.005

success = 0
while(runner.pos.z > catcher.pos.z):  ## Failure -> runner has reached the home plate
    rate(100)
    Fg = mass*g*vector(0, -1, 0) # gravity
    Fdrag = 0.5*density*Area* (mag( ball_p/mass - wind ))**2 * Cd * -norm(ball_p/mass - wind)  # air resistance
    v = mag(ball_p/mass - wind)
    Fmagnus = 0.5 * Cl * density * Area * v**2 * cross(norm(v_ang), norm(ball_p/mass - wind))  # Magnus force
    
    ## Momentum Principle
    ball_p += (Fg+Fdrag+Fmagnus)*dt
    ball.pos += (ball_p/mass)*dt
    runner.pos += runner_v*dt
    
    # ball bounces off ground
    if(ball.pos.y <= size and ball_p.y < 0):
        ball_p.y *= -COR
    # Success Condition
    if( ( (ball.pos.x-catcher.pos.x)**2 + (ball.pos.z-catcher.pos.z)**2 ) <= 1 and ball.pos.y <= 2 ):
        success = 1
        break
    
    t += dt
    
if(success): 
    scene.append_to_caption('YOU MADE IT!!!\n')
    text(text='YOU MADE IT!!!', align='center', depth=-6, billboard = True, color = vector((15*16+15)/255, (14*16+11)/255, (3*16+11)/255), emissive = 60, pos = field.pos + vector(0, 20, 0), height = 11, width = 120)
else:
    scene.append_to_caption('SORRY, YOU LOSE :(')
    text(text='SORRY, YOU LOSE :(', align='center', depth=-6, billboard = True, color = vector(0, (12*16+6)/255, (10*16+8)/255), emissive = 60, pos = field.pos + vector(0, 20, 0), height = 11, width = 120)

<IPython.core.display.Javascript object>

## Code: Part B
  ### Controlled Experiments -- placing the outfielder at a fixed spot 
 
 #### Experiment 1 -- How high to launch the ball?
Fixed **Launch speed** (and **spinrate**)    
Different **Head or tail wind (direct)** and **wind speeds**  
Altering **Launch angle** to see the results  
 

In [3]:
from vpython import *

## Define Constants

density = 1.225 # air density
Cd = 0.47 # drag coefficient of a sphere
pi = 3.1415926

g=9.8 # g = 9.8 m/s^2
size = 3.68e-2 # Baseball radius = 3.68 cm
mass = 0.145 # Baseball mass = 0.145 kg
height = 2.0 # Where the ball leaves your hand
Area = pi * (size**2) # effective surface area
density_ball = mass / (4/3 * size**3 * pi)

COR = 0.57 #Coefficient of Restitution of a baseball(regulated)
Cl = 0.22 # Lift coefficient (used in calculating Magnus force)


In [ ]:
# open the ballpark
scene = canvas(width=1000, height=400, center=vector(-20, 30, 20),background=vector(0,0,0))
scene.camera.pos = vector(-20, 100, 20)
scene.camera.axis = vector(20, -100, -20)
field = box(length=100, height=0.01, width=100, pos = vector(0, 0, 0), color=vector(0.5, 1, 0)) # the field
infield = box(length=40, height=0.01, width=40, pos = vector(30, 0.1, -30), color=vector(1, 0.7, 0.2)) # the infield
ball = sphere(radius = 3*size, color=color.white) # the baseball

# catcher at home plate
catcher = box(length=0.6, height=2, width=0.6, pos = vector(48, 1, -48), color = color.cyan)
# runner is set to cover 25 meters before scoring
runner = box(length=0.6, height=2, width=0.6, pos = vector(48, 1, -23), color = color.yellow)

# fixed outfielder position (80 meters away from catcher) -- you can change it if you want
you = box(length=0.6, height=2, width=0.6, pos = catcher.pos + 80*norm(vector(-1, 0, 1)), color = color.cyan)

## Initialize graph to show the statistics
graph1 = graph(width=800,height=400, xtitle = 'Windspeed (m/s)', ytitle = 'Launch angle (degree)')
gd_success = gdots(graph=graph1,size=3,color=color.green)
gd_successB = gdots(graph=graph1,size=3,color=color.blue)
gd_failure = gdots(graph=graph1,size=3,color=color.red)
gd_failureO = gdots(graph=graph1,size=3,color=color.black)

direction = norm(catcher.pos - you.pos)
wind_direction = direction
LaunchMPH = 90 # set default launch speed to 90mph -- you can change it if you want
Launch_v = LaunchMPH*1.609*5/18
spin_rate = 20*LaunchMPH + 400  #RPM (we used a linear regression formula)
v_ang = (spin_rate / 60 * 2 * pi) * norm(cross(vector(0, -1, 0), direction))  # the angular speed vector

scene.append_to_caption('Launch Speed = {:1d} mph\n'.format(LaunchMPH))
scene.append_to_caption('Runner {:1.1f} meters away\n'.format( runner.pos.z-catcher.pos.z ))
for w in range(-40, 41):
    wind = w/4 * wind_direction  ## windspeed -10 ~ 10 m/s (with 0.25 increment)
    
    for a in range(0, 46): ## Launch Angle 0 ~ 45 degree
        Launch_angle = a
        direction = norm( vector(direction.x, sqrt(direction.x**2 + direction.z**2)*tan(Launch_angle/180*pi), direction.z) )
        
        ball.pos = you.pos + vector(0, 1, 0)
        runner.pos = vector(48, 1, -23) ## re-place the runner after each iteration
        ball_p = mass*Launch_v*direction
        
        # default runner's speed = 7.5 m/s -- you can change it if you want
        runner_v = 7.5*norm( catcher.pos - runner.pos )
        t = 0
        dt = 0.005

        success = 0
        bounce = 0
        while(runner.pos.z > catcher.pos.z):  ## Failure -> runner has reached the home plate
            rate(50000)
            Fg = mass*g*vector(0, -1, 0) # gravity
            Fdrag = 0.5*density*Area* (mag( ball_p/mass - wind ))**2 * Cd * -norm(ball_p/mass - wind)  # air resistance
            v = mag(ball_p/mass - wind)
            Fmagnus = 0.5 * Cl * density * Area * v**2 * cross(norm(v_ang), norm(ball_p/mass - wind))  # Magnus force

            ## Momentum Principle
            ball_p += (Fg+Fdrag+Fmagnus)*dt
            ball.pos += (ball_p/mass)*dt
            runner.pos += runner_v*dt

            # ball bounces off ground
            if(ball.pos.y <= size and ball_p.y < 0):
                ball_p.y *= -COR
                bounce += 1
            # Success conditions
            if( ( (ball.pos.x-catcher.pos.x)**2 + (ball.pos.z-catcher.pos.z)**2 ) <= 1 and ball.pos.y <= 2 ):
                success = 1
                break

            t += dt

        if(success):
            if( bounce <= 2 ):  # less than 2 bounces -> Good!
                gd_success.plot([w/4, Launch_angle])
            else:
                gd_successB.plot([w/4, Launch_angle])
        else:
            if( ball.pos.z < -48 ):  ## cases of over-throwing
                gd_failureO.plot([w/4, Launch_angle])
            else:
                gd_failure.plot([w/4, Launch_angle])


<IPython.core.display.Javascript object>

#### Experiment 2 -- How much declination under crosswind?

Fixed **Launch speed** (and **spinrate**)    
Under **Complete crosswind** (perpendicular to the direction to catcher) and determined **wind speeds**  
Altering **Launch angle** and **Horizontal declination** to see the results 

In [1]:
from vpython import *

## Define Constants

density = 1.225 # air density
Cd = 0.47 # drag coefficient of a sphere
pi = 3.1415926

g=9.8 # g = 9.8 m/s^2
size = 3.68e-2 # Baseball radius = 3.68 cm
mass = 0.145 # Baseball mass = 0.145 kg
height = 2.0 # Where the ball leaves your hand
Area = pi * (size**2) # effective surface area
density_ball = mass / (4/3 * size**3 * pi)

COR = 0.57 #Coefficient of Restitution of a baseball(regulated)
Cl = 0.22 # Lift coefficient (used in calculating Magnus force)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# open the ballpark
scene = canvas(width=1000, height=400, center=vector(-20, 30, 20),background=vector(0,0,0))
scene.camera.pos = vector(-20, 100, 20)
scene.camera.axis = vector(20, -100, -20)
field = box(length=100, height=0.01, width=100, pos = vector(0, 0, 0), color=vector(0.5, 1, 0)) # the field
infield = box(length=40, height=0.01, width=40, pos = vector(30, 0.1, -30), color=vector(1, 0.7, 0.2)) # the infield
ball = sphere(radius = 3*size, color=color.white, make_trail=False) # the baseball

# catcher at home plate
catcher = box(length=0.6, height=2, width=0.6, pos = vector(48, 1, -48), color = color.cyan)
# runner is set to cover 25 meters before scoring -- you can change if you want (alter the z component)
runner = box(length=0.6, height=2, width=0.6, pos = vector(48, 1, -23), color = color.yellow)

# fixed outfielder position (80 meters away from catcher) -- you can change it if you want
you = box(length=0.6, height=2, width=0.6, pos = catcher.pos + 80*norm(vector(-1, 0, 1)), color = color.cyan)

## Initialize graph to show the statistics
graph1 = graph(width=800,height=400, xtitle = 'Horizontal declination (degree)', ytitle = 'Launch angle (degree)')
gd_success = gdots(graph=graph1,size=3,color=color.green)
gd_failureL = gdots(graph=graph1,size=3,color=color.red)
gd_failureR = gdots(graph=graph1,size=3,color=color.black)

direction = norm(catcher.pos - you.pos)
wind_direction = norm(cross(vector(0, -1, 0), direction)) ## crosswind coming from the left
LaunchMPH = 90 # set default launch speed to 90mph -- you can change it if you want
Launch_v = LaunchMPH*1.609*5/18
spin_rate = 20*LaunchMPH + 400  #RPM (we used a linear formula)
v_ang = (spin_rate / 60 * 2 * pi) * norm(cross(vector(0, -1, 0), direction))  # the angular speed vector

windspeed = 5 ## set windspeed = 5 m/s -- you can change if you want
wind = windspeed*wind_direction

scene.append_to_caption('Crosswind = {:1d} m/s\n'.format(windspeed))
scene.append_to_caption('Launch Speed = {:1d} mph\n'.format(LaunchMPH))
scene.append_to_caption('Runner {:1.1f} meters away\n'.format( runner.pos.z-catcher.pos.z ))
for dec in range(-100, 1): ## Hor declination -10 to 0 degree (with 0.1 increment)
    Hordec = dec/10
    
    for a in range(0, 46): ## Launch Angle 0 ~ 45 degree
        Launch_angle = a
        
        direction = norm(catcher.pos - you.pos) ## re-initialize direction
        ## use 2D rotation matrix to handle horizontal dec
        direction = vector( direction.x*cos(Hordec/180*pi)-direction.z*sin(Hordec/180*pi),
                           0, direction.x*sin(Hordec/180*pi)+direction.z*cos(Hordec/180*pi))
        ## adding vertical(y) component
        direction = norm( vector(direction.x, sqrt(direction.x**2 + direction.z**2)*tan(Launch_angle/180*pi), direction.z) )
        
        ball.pos = you.pos + vector(0, 1, 0) # re-place the runner after each iteration
        runner.pos = vector(48, 1, -23)
        ball_p = mass*Launch_v*direction
        
        # default runner's speed = 7.5 m/s -- you can change it if you want
        runner_v = 7.5*norm( catcher.pos - runner.pos )
        t = 0
        dt = 0.005

        success = 0
        while(runner.pos.z > catcher.pos.z):  ## Failure -> runner has reached the home plate
            rate(50000)
            Fg = mass*g*vector(0, -1, 0) # gravity
            Fdrag = 0.5*density*Area* (mag( ball_p/mass - wind ))**2 * Cd * -norm(ball_p/mass - wind)  # air resistance
            v = mag(ball_p/mass - wind)
            Fmagnus = 0.5 * Cl * density * Area * v**2 * cross(norm(v_ang), norm(ball_p/mass - wind))  # Magnus force

            ## Momentum Principle
            ball_p += (Fg+Fdrag+Fmagnus)*dt
            ball.pos += (ball_p/mass)*dt
            runner.pos += runner_v*dt

            # ball bounces off ground
            if(ball.pos.y <= size and ball_p.y < 0):
                ball_p.y *= -COR
            # Success conditions
            if( ( (ball.pos.x-catcher.pos.x)**2 + (ball.pos.z-catcher.pos.z)**2 ) <= 1 and ball.pos.y <= 2 ):
                success = 1
                break

            t += dt

        if(success): 
            gd_success.plot([Hordec, Launch_angle])
        else:
            if( ball.pos.z+ball.pos.x < 0 ):  ## missing left
                gd_failureL.plot([Hordec, Launch_angle])
            else:                             ## missing right
                gd_failureR.plot([Hordec, Launch_angle])


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

 ## --- above is last semester --
 
 # Part I: DC Current

In [18]:
from vpython import *

## define constants
mu0_over4pi = 1e-7
pi = acos(-1.0)
R = 0.15
I = 50
qe = 1.602e-19
mass_e = 9.11e-31
c = 3e8

##extrusion(path=paths.circle(pos=vec(0.475*apart,0,0),radius=R*1.1,up=vec(1,0,0)), texture=textures.metal, 
#    shape=[[shapes.rectangle(pos=[0,0.1*R,0],width=0.01,height=0.003)], 
#    [shapes.rectangle(pos=[0.005,0.1*R,0],width=0.001,height=0.01)], 
#    [shapes.rectangle(pos=[-0.005,0.1*R,0],width=0.001,height=0.01)]])
##cylinder(pos=vector(0.5*apart-0.05*R, 0, 0), axis = vector(0.1*R, 0, 0), radius = R,
##             color = color.yellow, shininess = 0.6)

In [ ]:
enviro = canvas(width=600, height=400, title = 'click to shoot electron in')
apart = R

## Build the coils

I_ring1 = extrusion(path=paths.circle(pos=vec(0.5*apart,0,0),radius=R*1.1,up=vec(1,0,0)),
                   texture=textures.metal, color = color.yellow,
                    shape=[[shapes.rectangle(pos=[0,0.1*R,0],width=0.01,height=0.003)], 
                            [shapes.rectangle(pos=[0.005,0.1*R,0],width=0.001,height=0.01)], 
                            [shapes.rectangle(pos=[-0.005,0.1*R,0],width=0.001,height=0.01)]])

con_rod = cylinder(pos=vector(0.5*R, -R, 0), axis = vector(0, -0.2*R, 0), radius = 0.04*R, color = color.gray(0.8), shininess = 0.6)
base = box(pos=vector(0.5*R, -1.3*R, 0), size=vector(0.6*R, 0.3*R, 0.6*R), color = vector(1, 0.7, 0.2))

coil1 = compound([con_rod, base])
coil2 = coil1.clone(pos = vec(-coil1.pos.x, coil1.pos.y, 0))
I_ring2 = I_ring1.clone(pos = -I_ring1.pos)


## discretize the coils
segment_Y = []
segment_Z = []
for i in range(0, 73):
    segment_Y.append( R*cos(i*2*pi/72) )
    segment_Z.append( R*sin(i*2*pi/72) )

## For x-axis field plotting
x_field = graph(width=800,height=400, xtitle = 'x_position (*R)', ytitle = 'Magnetic field (T)', ymax = 4e-4)
gc_B1 = gcurve(graph=x_field,color=color.green)
gc_B2 = gcurve(graph=x_field,color=color.blue)
gc_B = gcurve(graph=x_field,color=color.magenta)

def x_plot(dis):
    gc_B1.delete()
    gc_B2.delete()
    gc_B.delete()
    segment_pos = []
    dLength = []
    
    for i in range(0, 72):
        segment_pos.append( 0.5*vec(0.5*dis, segment_Y[i], segment_Z[i]) + 
                            0.5*vec(0.5*dis, segment_Y[i+1], segment_Z[i+1]) )
        dLength.append( vec(0.5*dis, segment_Y[i+1], segment_Z[i+1]) - 
                        vec(0.5*dis, segment_Y[i], segment_Z[i]) )

    ## compute B-field along x-axis
    for i in range(-200, 201):
        now_pos = vec(i*(dis/200),0,0)
        B_now = vec(0,0,0)
        B1 = B2 = vec(0, 0, 0)
        
        for j in range (0,72):
            pos1 = segment_pos[j]
            pos2 = vec(-pos1.x, pos1.y, pos1.z)
            dB1 = mu0_over4pi*I*cross(dLength[j], now_pos - pos1) / mag(now_pos - pos1)**3
            dB2 = mu0_over4pi*I*cross(dLength[j], now_pos - pos2) / mag(now_pos - pos2)**3
            B1 += dB1
            B2 += dB2
            B_now += dB1+dB2

        gc_B.plot([now_pos.x/R, mag(B_now)])
        gc_B1.plot([now_pos.x/R, mag(B1)])
        gc_B2.plot([now_pos.x/R, mag(B2)])
    
enviro.caption = "Vary the distance between coils\n"
def setapart(s):
    wt.text = '{:1.2f}'.format(s.value)
    global apart
    apart = s.value*R
    coil1.pos.x = 0.5*apart
    coil2.pos.x = -0.5*apart
    I_ring1.pos.x = 0.5*apart
    I_ring2.pos.x = -0.5*apart
    x_plot(apart)
    
sl_ap = slider(min=0.5, max=3, value=1, length=220, bind=setapart, right=15)
wt = wtext(text='{:1.2f}'.format(sl_ap.value))
enviro.append_to_caption('* Radius of coil\n')

def B_calculate(pos):
    B_now = vec(0,0,0)
    for j in range (0,72):
        pos1 = segment_pos[j]
        pos2 = vec(-pos1.x, pos1.y, pos1.z)
        dB1 = mu0_over4pi*I*cross(dLength[j], pos - pos1) / mag(pos - pos1)**3
        dB2 = mu0_over4pi*I*cross(dLength[j], pos - pos2) / mag(pos - pos2)**3
        B_now += dB1+dB2
    return B_now

## send an electron in
electron = sphere(pos=vec(0, 0, 0), radius = 0.05*R, make_trail = True)
enviro.waitfor('click')

t = 0
dt = 1e-10
e_velo = vec(0, -8e-3*c, 0)
while(mag(electron.pos) <= 5*R):
    rate(1000000)
    Fmag = -qe*cross(e_velo, B_calculate(electron.pos))
    e_velo += Fmag/mass_e*dt
    electron.pos += e_velo*dt
    t += dt


<IPython.core.display.Javascript object>